In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.autograd import Variable
from classification_models import GetLoader

In [2]:
class CausalConv1d(nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size=2, stride=1,
                 padding=1, dilation=1, groups=1, bias=False):
        super(CausalConv1d, self).__init__(in_channels, out_channels, kernel_size, stride,
                                           padding, dilation, groups, bias)
    
    def forward(self, inputs):
        outputs = super(CausalConv1d, self).forward(inputs)
        return outputs[:,:,:-1]

In [3]:
class DilatedConv1d(nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size=2, stride=1,
                 padding=0, dilation=1, groups=1, bias=False):
        super(DilatedConv1d, self).__init__(in_channels, out_channels, kernel_size, stride,
                                            padding, dilation, groups, bias)
    
    def forward(self, inputs):
        outputs = super(DilatedConv1d, self).forward(inputs)
        return outputs

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, res_channels, skip_channels, dilation):
        super(ResidualBlock, self).__init__()
        self.filter_conv = DilatedConv1d(in_channels=res_channels, out_channels=res_channels, dilation=dilation)
        self.gate_conv = DilatedConv1d(in_channels=res_channels, out_channels=res_channels, dilation=dilation)
        self.skip_conv = nn.Conv1d(in_channels=res_channels, out_channels=skip_channels, kernel_size=1)
        self.residual_conv = nn.Conv1d(in_channels=res_channels, out_channels=res_channels, kernel_size=1)
        
    def forward(self,inputs):
        #print(dilation)
        sigmoid_out = torch.sigmoid(self.gate_conv(inputs))
        tahn_out = torch.tanh(self.filter_conv(inputs))
        output = sigmoid_out * tahn_out
        #
        skip_out = self.skip_conv(output)
        res_out = self.residual_conv(output)
        res_out = res_out + inputs[:, :, -res_out.size(2):]
        # res
        return res_out , skip_out

In [5]:
class WaveNet(nn.Module):
    def __init__(self, num_classes, in_depth=128, in_channels=42, skip_channels=64, dilation_depth=7, device=torch.device("cpu")):
        super(WaveNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=32,
                               kernel_size=2, stride=1, padding=1, bias=False)
        
        self.dilations = [2**i for i in range(dilation_depth)]
        self.main = nn.ModuleList([ResidualBlock(res_channels=32, 
                                                 skip_channels=skip_channels,dilation=dilation) for dilation in self.dilations])
        #self.pre = nn.Embedding(in_depth, res_channels)
        #self.pre_conv = CausalConv1d(in_channels=res_channels, out_channels=res_channels)
        self.post = nn.Sequential(nn.ReLU(),
                                  nn.Conv1d(skip_channels,skip_channels,1),
                                  nn.ReLU(),
                                  nn.Conv1d(skip_channels,in_depth,kernel_size=1))
        self.fc = nn.Linear(in_depth*2, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight.to(device), mode='fan_out', nonlinearity='relu')
        
    def forward(self,inputs):
        #outputs = self.preprocess(inputs)
        skip_connections = []
        
        outputs = self.conv1(inputs)
        
        for layer in self.main:
            outputs,skip = layer(outputs)
            skip_connections.append(skip)
        #print(outputs.shape)
        outputs = sum([s[:,:,-outputs.size(2):] for s in skip_connections])
        #print("shape")
        outputs = self.post(outputs)
        #print(outputs.shape)  
        outputs = torch.flatten(outputs, 1)
        #print(outputs.shape)
        outputs = self.fc(outputs)
        
        return outputs

In [6]:
def predict_precision(model,images,labels,device,predict_type):
    model.eval()
    with torch.no_grad():
        correct=0
        total=0
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total+=sum(predicted)
        correct+=(sum(predicted*labels))
        print('precision of the model on the'+predict_type+'data: {}%'.format(100*correct/total))
    model.train()
    return predicted, 100*correct/total

In [7]:
#导入train数据
a=np.load("train_data.npy")
b=a.reshape([a.shape[0],a.shape[1]*a.shape[2]])
c=b[~np.isnan(b).any(axis=1),:]
train_data=c.reshape([c.shape[0], a.shape[1],a.shape[2]])
d=np.load("train_label.npy").reshape(-1,1)
train_label=d[~np.isnan(b).any(axis=1),:].reshape(-1,)

In [8]:
#导入valid数据
a=np.load("valid_data.npy")
b=a.reshape([a.shape[0],a.shape[1]*a.shape[2]])
c=b[~np.isnan(b).any(axis=1),:]
valid_data=c.reshape([c.shape[0], a.shape[1],a.shape[2]])
d=np.load("valid_label.npy").reshape(-1,1)
valid_label=d[~np.isnan(b).any(axis=1),:].reshape(-1,)

In [9]:
train_data = np.transpose(train_data,(0,2,1))
#train_data = train_data[:,:,0:9]
valid_data = np.transpose(valid_data,(0,2,1)) 
#valid_data = valid_data[:,:,0:9]

In [10]:
train_data.shape

(42531, 42, 10)

In [11]:
#wavenet trial 1: 尝试在旧的10日为sequence的数据上实验
num_classes=2 
in_depth=64 
in_channels=42 
skip_channels=32 
dilation_depth=3 
device= torch.device("cuda:1")
batch_size=128
learning_rate=0.001
num_epochs=1000

In [12]:
train=GetLoader(train_data,train_label)
valid=GetLoader(valid_data,valid_label)
train_loader=torch.utils.data.DataLoader(dataset=train,batch_size=batch_size,shuffle=True,num_workers=0)
valid_loader=torch.utils.data.DataLoader(dataset=valid,batch_size=valid_data.shape[0],shuffle=False,num_workers=0)

In [13]:
wavenet = WaveNet(num_classes, in_depth, in_channels, skip_channels, dilation_depth, device)
wavenet.to(device)

WaveNet(
  (conv1): Conv1d(42, 32, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
  (main): ModuleList(
    (0): ResidualBlock(
      (filter_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), bias=False)
      (gate_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), bias=False)
      (skip_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      (residual_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (1): ResidualBlock(
      (filter_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(2,), bias=False)
      (gate_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(2,), bias=False)
      (skip_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      (residual_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (2): ResidualBlock(
      (filter_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(4,), bias=False)
      (gate_conv): DilatedConv1d(32, 32, kernel_size=(2,), stri

In [14]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(wavenet.parameters(),lr=learning_rate)

In [15]:
total_step=0
train_precision=[]
valid_precision=[]

In [ ]:
#训练过程
for epoch in range(num_epochs):
    for data in train_loader:
        images, labels = data
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        #forward pass
        outputs=wavenet(images)
        loss=loss_function(outputs,labels.long())
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_step+=1
        if (total_step)%1000==0:#each 10 iterations is one epoch
            print("Epoch [{}/{}],step[{}] Loss:{:.4f}".format(epoch+1,num_epochs,total_step,loss.item()))
            _,train_pre=predict_precision(wavenet,images,labels,device,predict_type='training')
            train_precision.append(train_pre)
            for images, labels in valid_loader:
                _,valid_pre=predict_precision(wavenet,images,labels,device,predict_type='validation')
            valid_precision.append(valid_pre)

In [ ]:
for data in valid_loader:
    images, labels = data
    predict_precision(wavenet,images,labels,device,'train')

In [ ]:
plt.plot(train_precision,label="training precision")
plt.plot(valid_precision,label="validation precision")
plt.title("WaveNet trial 1")
plt.xlabel("1000*x training step")
plt.ylabel("precision")

In [ ]:
#wavenet trial 1：效果不佳
#下一步实验：清洗出正好符合结构的sequence=8的数据进行试验

In [16]:
#wavenet trial 2：
#导入train数据
a=np.load("wave_train_data.npy")
b=a.reshape([a.shape[0],a.shape[1]*a.shape[2]])
c=b[~np.isnan(b).any(axis=1),:]
train_data=c.reshape([c.shape[0], a.shape[1],a.shape[2]])
d=np.load("wave_train_label.npy").reshape(-1,1)
train_label=d[~np.isnan(b).any(axis=1),:].reshape(-1,)
#导入valid数据
a=np.load("wave_valid_data.npy")
b=a.reshape([a.shape[0],a.shape[1]*a.shape[2]])
c=b[~np.isnan(b).any(axis=1),:]
valid_data=c.reshape([c.shape[0], a.shape[1],a.shape[2]])
d=np.load("wave_valid_label.npy").reshape(-1,1)
valid_label=d[~np.isnan(b).any(axis=1),:].reshape(-1,)

In [17]:
train_data = np.transpose(train_data,(0,2,1))
valid_data = np.transpose(valid_data,(0,2,1)) 

In [18]:
train_data.shape, valid_data.shape

((49745, 42, 8), (21317, 42, 8))

In [19]:
num_classes=2 
in_depth=128 
in_channels=42 
skip_channels=64 
dilation_depth=3 
device= torch.device("cuda:1")
batch_size=128
learning_rate=0.001
num_epochs=1000

In [20]:
train=GetLoader(train_data,train_label)
valid=GetLoader(valid_data,valid_label)
train_loader=torch.utils.data.DataLoader(dataset=train,batch_size=batch_size,shuffle=True,num_workers=0)
valid_loader=torch.utils.data.DataLoader(dataset=valid,batch_size=valid_data.shape[0],shuffle=False,num_workers=0)

In [21]:
wavenet = WaveNet(num_classes, in_depth, in_channels, skip_channels, dilation_depth, device)
wavenet.to(device)

WaveNet(
  (conv1): Conv1d(42, 32, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
  (main): ModuleList(
    (0): ResidualBlock(
      (filter_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), bias=False)
      (gate_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), bias=False)
      (skip_conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
      (residual_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (1): ResidualBlock(
      (filter_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(2,), bias=False)
      (gate_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(2,), bias=False)
      (skip_conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
      (residual_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (2): ResidualBlock(
      (filter_conv): DilatedConv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(4,), bias=False)
      (gate_conv): DilatedConv1d(32, 32, kernel_size=(2,), stri

In [22]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(wavenet.parameters(),lr=learning_rate)

In [23]:
total_step=0
train_precision=[]
valid_precision=[]

In [24]:
#训练过程
for epoch in range(num_epochs):
    for data in train_loader:
        images, labels = data
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        #forward pass
        outputs=wavenet(images)
        loss=loss_function(outputs,labels.long())
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_step+=1
        if total_step>90*1000:
            break
        if (total_step)%1000==0:#each 10 iterations is one epoch
            print("Epoch [{}/{}],step[{}] Loss:{:.4f}".format(epoch+1,num_epochs,total_step,loss.item()))
            _,train_pre=predict_precision(wavenet,images,labels,device,predict_type='training')
            train_precision.append(train_pre)
            for images, labels in valid_loader:
                _,valid_pre=predict_precision(wavenet,images,labels,device,predict_type='validation')
            valid_precision.append(valid_pre)

Epoch [3/1000],step[1000] Loss:0.6792
precision of the model on thetrainingdata: 40.0%
precision of the model on thevalidationdata: 55.72289276123047%
Epoch [6/1000],step[2000] Loss:0.6774
precision of the model on thetrainingdata: 55.319149017333984%
precision of the model on thevalidationdata: 52.509124755859375%
Epoch [8/1000],step[3000] Loss:0.6684
precision of the model on thetrainingdata: 68.18181610107422%
precision of the model on thevalidationdata: 54.66666793823242%
Epoch [11/1000],step[4000] Loss:0.6799
precision of the model on thetrainingdata: 54.761905670166016%
precision of the model on thevalidationdata: 56.9331169128418%
Epoch [13/1000],step[5000] Loss:0.6779
precision of the model on thetrainingdata: 64.81481170654297%
precision of the model on thevalidationdata: 57.079647064208984%
Epoch [16/1000],step[6000] Loss:0.6604
precision of the model on thetrainingdata: 44.44444274902344%
precision of the model on thevalidationdata: 55.241458892822266%
Epoch [18/1000],step[7

precision of the model on thevalidationdata: 55.0%
Epoch [134/1000],step[52000] Loss:0.5701
precision of the model on thetrainingdata: 73.91304016113281%
precision of the model on thevalidationdata: 55.524078369140625%
Epoch [137/1000],step[53000] Loss:0.5927
precision of the model on thetrainingdata: 85.71428680419922%
precision of the model on thevalidationdata: 56.10200500488281%
Epoch [139/1000],step[54000] Loss:0.5008
precision of the model on thetrainingdata: 69.76744079589844%
precision of the model on thevalidationdata: 55.42671585083008%
Epoch [142/1000],step[55000] Loss:0.5732
precision of the model on thetrainingdata: 76.08695983886719%
precision of the model on thevalidationdata: 55.31219482421875%
Epoch [144/1000],step[56000] Loss:0.5361
precision of the model on thetrainingdata: 75.0%
precision of the model on thevalidationdata: 54.73684310913086%
Epoch [147/1000],step[57000] Loss:0.5180
precision of the model on thetrainingdata: 82.05128479003906%
precision of the model 

In [ ]:
plt.plot(train_precision,label="training precision")
plt.plot(valid_precision,label="validation precision")
plt.title("WaveNet trial 2")
plt.xlabel("1000*x training step")
plt.ylabel("precision")

In [ ]:
#wavenet trial 2:表现稳定在precision 0.55-0.58，但还有gradient vanishing问题没有解决
#下一步试验：调整参数解决gradient vanishing

In [ ]:
#wavenet trial 3：
num_classes=2 
in_depth=64 
in_channels=42 
skip_channels=64 
dilation_depth=3 
device= torch.device("cuda:1")
batch_size=128
learning_rate=0.0001
num_epochs=1000

In [ ]:
train=GetLoader(train_data,train_label)
valid=GetLoader(valid_data,valid_label)
train_loader=torch.utils.data.DataLoader(dataset=train,batch_size=batch_size,shuffle=True,num_workers=0)
valid_loader=torch.utils.data.DataLoader(dataset=valid,batch_size=valid_data.shape[0],shuffle=False,num_workers=0)

In [ ]:
wavenet = WaveNet(num_classes, in_depth, in_channels, skip_channels, dilation_depth, device)
wavenet.to(device)

In [ ]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(wavenet.parameters(),lr=learning_rate)

In [ ]:
total_step=0
train_precision=[]
valid_precision=[]

In [ ]:
#训练过程
for epoch in range(num_epochs):
    for data in train_loader:
        images, labels = data
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        #forward pass
        outputs=wavenet(images)
        loss=loss_function(outputs,labels.long())
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_step+=1
        if (total_step)%1000==0:#each 10 iterations is one epoch
            print("Epoch [{}/{}],step[{}] Loss:{:.4f}".format(epoch+1,num_epochs,total_step,loss.item()))
            _,train_pre=predict_precision(wavenet,images,labels,device,predict_type='training')
            train_precision.append(train_pre)
            for images, labels in valid_loader:
                _,valid_pre=predict_precision(wavenet,images,labels,device,predict_type='validation')
            valid_precision.append(valid_pre)

In [ ]:
plt.plot(train_precision,label="training precision")
plt.plot(valid_precision,label="validation precision")
plt.title("WaveNet trial 3")
plt.xlabel("1000*x training step")
plt.ylabel("precision")

In [ ]:
#wavenet trial 3:将最终的conv channel数减少，有一定的效果
#下一步试验：增加bacth_norm

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, res_channels, skip_channels, dilation):
        super(ResidualBlock, self).__init__()
        self.filter_conv = DilatedConv1d(in_channels=res_channels, out_channels=res_channels, dilation=dilation)
        self.gate_conv = DilatedConv1d(in_channels=res_channels, out_channels=res_channels, dilation=dilation)
        self.skip_conv = nn.Conv1d(in_channels=res_channels, out_channels=skip_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(res_channels)
        self.residual_conv = nn.Conv1d(in_channels=res_channels, out_channels=res_channels, kernel_size=1)
        
    def forward(self,inputs):
        #print(dilation)
        inputs = self.bn1(inputs)
        sigmoid_out = torch.sigmoid(self.gate_conv(inputs))
        tahn_out = torch.tanh(self.filter_conv(inputs))
        output = sigmoid_out * tahn_out
        #
        skip_out = self.skip_conv(output)
        res_out = self.residual_conv(output)
        res_out = res_out + inputs[:, :, -res_out.size(2):]
        # res
        return res_out , skip_out
    


class WaveNet(nn.Module):
    def __init__(self, num_classes, in_depth=128, in_channels=42, skip_channels=64, dilation_depth=7, device=torch.device("cpu")):
        super(WaveNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=32,
                               kernel_size=2, stride=1, padding=1, bias=False)
        
        self.dilations = [2**i for i in range(dilation_depth)]
        self.main = nn.ModuleList([ResidualBlock(res_channels=32, 
                                                 skip_channels=skip_channels,dilation=dilation) for dilation in self.dilations])
        #self.pre = nn.Embedding(in_depth, res_channels)
        #self.pre_conv = CausalConv1d(in_channels=res_channels, out_channels=res_channels)
        self.post = nn.Sequential(nn.BatchNorm1d(skip_channels),
                                  nn.ReLU(),
                                  nn.Conv1d(skip_channels,skip_channels,1),
                                  nn.BatchNorm1d(skip_channels),
                                  nn.ReLU(),
                                  nn.Conv1d(skip_channels,in_depth,kernel_size=1))
        self.fc = nn.Linear(in_depth*2, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight.to(device), mode='fan_out', nonlinearity='relu')
        
    def forward(self,inputs):
        #outputs = self.preprocess(inputs)
        skip_connections = []
        
        outputs = self.conv1(inputs)
        
        for layer in self.main:
            outputs,skip = layer(outputs)
            skip_connections.append(skip)
        #print(outputs.shape)
        outputs = sum([s[:,:,-outputs.size(2):] for s in skip_connections])
        #print("shape")
        outputs = self.post(outputs)
        #print(outputs.shape)  
        outputs = torch.flatten(outputs, 1)
        #print(outputs.shape)
        outputs = self.fc(outputs)
        
        return outputs

In [ ]:
wavenet = WaveNet(num_classes, in_depth, in_channels, skip_channels, dilation_depth, device)
wavenet.to(device)

In [ ]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(wavenet.parameters(),lr=learning_rate)

In [ ]:
total_step=0
train_precision=[]
valid_precision=[]

In [ ]:
#训练过程
for epoch in range(num_epochs):
    for data in train_loader:
        images, labels = data
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        #forward pass
        outputs=wavenet(images)
        loss=loss_function(outputs,labels.long())
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_step+=1
        if (total_step)%1000==0:#each 10 iterations is one epoch
            print("Epoch [{}/{}],step[{}] Loss:{:.4f}".format(epoch+1,num_epochs,total_step,loss.item()))
            _,train_pre=predict_precision(wavenet,images,labels,device,predict_type='training')
            train_precision.append(train_pre)
            for images, labels in valid_loader:
                _,valid_pre=predict_precision(wavenet,images,labels,device,predict_type='validation')
            valid_precision.append(valid_pre)

In [ ]:
plt.plot(train_precision,label="training precision")
plt.plot(valid_precision,label="validation precision")
plt.title("WaveNet trial 4")
plt.xlabel("1000*x training step")
plt.ylabel("precision")

In [ ]:
#结论：batch_norm不管用

In [27]:
for images, labels in valid_loader:
    with torch.no_grad():
        correct=0
        total=0
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        outputs=wavenet(images)
        _,predicted=torch.max(outputs.data,1)
        softmax=nn.Softmax(-1)
        outputs=softmax(outputs)
        total+=sum(predicted)
        correct+=(sum(predicted*labels))
        print('precision of the model on the'+'validation'+'data: {}%'.format(100*correct/total))

precision of the model on thevalidationdata: 55.79344940185547%


In [30]:
torch.max(outputs,1)

torch.return_types.max(
values=tensor([0.5932, 0.5932, 0.5383,  ..., 0.5721, 0.5505, 0.5702], device='cuda:1'),
indices=tensor([0, 0, 1,  ..., 0, 0, 0], device='cuda:1'))

In [34]:
s=nn.Softmax(-1)
s(outputs)

tensor([[0.5465, 0.4535],
        [0.5465, 0.4535],
        [0.4808, 0.5192],
        ...,
        [0.5360, 0.4640],
        [0.5252, 0.4748],
        [0.5350, 0.4650]], device='cuda:1')